In [ ]:
import numpy as np
from tensorflow.keras.utils import Sequence
from tensorflow.keras.utils import to_categorical
def create_adjacency_matrix(sequence_length):
    adjacency_matrix = np.eye(sequence_length, k=1) + np.eye(sequence_length, k=-1)
    return adjacency_matrix

#here the sequence generator is different because an adjacency matrix has to be created in order for the GCN model to find connections between nodes

class SequentialDataGenerator(Sequence):
    def __init__(self, X, Y_int, batch_size=16, num_classes=5, max_sequence_length=None):
        self.X = X
        self.Y_int = Y_int
        self.batch_size = batch_size
        self.num_classes = num_classes
        self.max_sequence_length = max_sequence_length or self._max_length()
        self.indexes = np.arange(len(self.X))

    def __len__(self):
        return int(np.ceil(len(self.X) / self.batch_size))

    def __getitem__(self, index):
        indexes = self.indexes[index * self.batch_size:(index + 1) * self.batch_size]

        batch_X = [self.X[k] for k in indexes]
        batch_Y_int = [self.Y_int[k] for k in indexes]

        X_temp_padded = self._pad_sequences(batch_X, self.max_sequence_length)

        Y_temp_padded = self._process_labels(batch_Y_int)

        batch_A = np.array([create_adjacency_matrix(self.max_sequence_length) for _ in range(len(indexes))])

        return [np.array(X_temp_padded), batch_A], Y_temp_padded
    def on_epoch_end(self):
        pass

    def _max_length(self):
        return max(len(seq) for seq in self.X)

    def _pad_sequences(self, sequences, max_seq_length, pad_val=0):
        padded_sequences = []
        for seq in sequences:
            seq = np.array(seq)
            pad_size = max_seq_length - len(seq)

            if pad_size > 0:
                pad_shape = list(seq.shape)
                pad_shape[0] = pad_size

                pad_array = np.full(pad_shape, pad_val)

                padded_seq = np.concatenate((seq, pad_array), axis=0)
            else:
                padded_seq = seq

            padded_sequences.append(padded_seq)

        return np.array(padded_sequences)

    def _process_labels(self, labels):
        Y_temp_padded = np.zeros((len(labels), self.max_sequence_length, self.max_sequence_length, self.num_classes))

        for i, seq_labels in enumerate(labels):
            one_hot_labels = to_categorical(seq_labels, num_classes=self.num_classes)
            pad_len = self.max_sequence_length - len(one_hot_labels)
            if pad_len > 0:
                one_hot_labels = np.vstack((one_hot_labels, np.zeros((pad_len, self.num_classes))))


            Y_temp_padded[i, :len(one_hot_labels), :len(one_hot_labels), :] = one_hot_labels

        return Y_temp_padded

In [ ]:
num_classes = 5
num_nodes = 16 #arbritary number of nodes


train_generator = SequentialDataGenerator(X_train, Y_train_int, batch_size=16, num_classes=num_classes)

validation_generator = SequentialDataGenerator(X_test, Y_test_int, batch_size=16, num_classes=num_classes)

train_X, train_Y = next(iter(train_generator))
print("Training batch X shape:", train_X[0].shape)
print("Training batch adjacency matrix shape:", train_X[1].shape)
print("Training batch Y shape:", train_Y.shape)

validation_X, validation_Y = next(iter(validation_generator))
print("Validation batch X shape:", validation_X[0].shape)
print("Validation batch adjacency matrix shape:", validation_X[1].shape)
print("Validation batch Y shape:", validation_Y.shape)

In [ ]:
pip install spektral

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import TimeDistributed, Dense, Flatten, Dropout, Layer, Input, Reshape, GlobalAveragePooling1D
from tensorflow.keras.models import Model
from tensorflow.keras.regularizers import l2
from tensorflow.keras.applications.efficientnet_v2 import EfficientNetV2S
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import layers, models
from spektral.layers import GCNConv

def create_sequence_model(sequence_length, num_nodes, num_features, num_classes, dropout_rate=0.25, l2_reg=0.001):
    #weights frozen for base
    cnn_base = EfficientNetV2S(include_top=False, weights='imagenet', pooling='avg')
    cnn_base.trainable = False
    video_input = layers.Input(shape=(sequence_length, 224, 224, 3))
    adjacency_input = layers.Input(shape=(num_nodes, num_nodes))
    cnn_out = TimeDistributed(cnn_base)(video_input)
    cnn_out = Dropout(dropout_rate)(cnn_out)
    adjacency_input_reshaped = Reshape((num_nodes, num_nodes))(adjacency_input)


    gnn_out = GCNConv(64, activation='relu', kernel_regularizer=l2(l2_reg))([cnn_out, adjacency_input_reshaped])
    gnn_out = GlobalAveragePooling1D()(gnn_out)

    output = Dense(sequence_length * num_nodes * num_classes, activation='softmax', kernel_regularizer=l2(l2_reg))(gnn_out)  # Reduced L2 regularization
    output = Reshape((sequence_length, num_nodes, num_classes))(output)
    output = layers.Permute((2, 1, 3))(output)  # Swap dimensions to match label shape

    model = models.Model(inputs=[video_input, adjacency_input], outputs=output)

    optimizer = Adam(learning_rate=0.0001)
    model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

    return model

model = create_sequence_model(sequence_length=16, num_nodes=16, num_features=1280, num_classes=5)
model.summary()

In [ ]:
history = model.fit(
    train_generator,
    epochs= 200,
    validation_data=validation_generator
)